In [2]:

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

True

In [3]:

from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [5]:
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile()

In [6]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

chatbot.invoke(initial_state)['messages'][-1].content

'The capital of India is **New Delhi**.'

# Chatbot Loop

In [7]:
while True:
    user_input = input("You: ")
    print("User:",user_input)
    if user_input.strip().lower() in ["exit",'quit',"bye"]:
        break
    response=chatbot.invoke({'messages':HumanMessage(content=user_input)})
    print("Bot:",response['messages'][-1].content)

User: Hi I'm aditya
Bot: Hi Aditya! It's nice to meet you. How can I help you today?
User: what is 1+1
Bot: 1 + 1 = 2
User: can you add 10 to the result
Bot: Okay, I need a little more information! To add 10 to the result, I need to know what the result *is*. 

Please tell me:

*   **What result are you referring to?** For example, is it the result of a calculation, a previous answer I gave, or something else?

Once you tell me what the "result" is, I can easily add 10 to it.
User: exit
